# Attribution Patching

📗 This tutorial is adapted from Neel Nanda's blog post, [here](https://www.neelnanda.io/mechanistic-interpretability/attribution-patching).

📘 Attribution patching was used in [Attribution Patching Outperforms Automated Circuit Discovery](https://arxiv.org/abs/2310.10348).

📙 You can find a colab version of the tutorial here, or Neel's version [here](https://colab.research.google.com/github/neelnanda-io/TransformerLens/blob/main/demos/Attribution_Patching_Demo.ipynb).

Attribution patching is a technique that uses gradient-based approximation to approximate activation patching. Attribution patching allows you to do every single patch you might want between a clean and corrupted input on two forward passes and one backward pass. That is, every single patch can be calculated from the results of just 3 runs!

# Setup (Ignore)

In [17]:
import einops
import torch
import plotly.express as px

from nnsight import LanguageModel

# 1️⃣ IOI Patching Setup

In [18]:
model = LanguageModel("openai-community/gpt2", device_map="auto", dispatch=True)

In [19]:
prompts = [
    "When John and Mary went to the shops, John gave the bag to",
    "When John and Mary went to the shops, Mary gave the bag to",
    "When Tom and James went to the park, James gave the ball to",
    "When Tom and James went to the park, Tom gave the ball to",
    "When Dan and Sid went to the shops, Sid gave an apple to",
    "When Dan and Sid went to the shops, Dan gave an apple to",
    "After Martin and Amy went to the park, Amy gave a drink to",
    "After Martin and Amy went to the park, Martin gave a drink to",
]

answers = [
    (" Mary", " John"),
    (" John", " Mary"),
    (" Tom", " James"),
    (" James", " Tom"),
    (" Dan", " Sid"),
    (" Sid", " Dan"),
    (" Martin", " Amy"),
    (" Amy", " Martin"),
]

clean_tokens = model.tokenizer(prompts, return_tensors="pt")["input_ids"]

corrupted_tokens = clean_tokens[
    [(i + 1 if i % 2 == 0 else i - 1) for i in range(len(clean_tokens))]
]

answer_token_indices = torch.tensor(
    [
        [model.tokenizer(answers[i][j])["input_ids"][0] for j in range(2)]
        for i in range(len(answers))
    ]
)

In [20]:
def get_logit_diff(logits, answer_token_indices=answer_token_indices):
    if len(logits.shape) == 3:
        # Get final logits only
        logits = logits[:, -1, :]
    correct_logits = logits.gather(1, answer_token_indices[:, 0].unsqueeze(1))
    incorrect_logits = logits.gather(1, answer_token_indices[:, 1].unsqueeze(1))
    return (correct_logits - incorrect_logits).mean()

clean_logits = model.trace(clean_tokens, trace=False).logits.cpu()
corrupted_logits = model.trace(corrupted_tokens, trace=False).logits.cpu()

CLEAN_BASELINE = get_logit_diff(clean_logits, answer_token_indices).item()
print(f"Clean logit diff: {CLEAN_BASELINE:.4f}")

CORRUPTED_BASELINE = get_logit_diff(corrupted_logits, answer_token_indices).item()
print(f"Corrupted logit diff: {CORRUPTED_BASELINE:.4f}")

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Clean logit diff: 2.8138
Corrupted logit diff: -2.8138


In [21]:
def ioi_metric(
    logits,
    answer_token_indices=answer_token_indices,
):
    return (get_logit_diff(logits, answer_token_indices) - CORRUPTED_BASELINE) / (
        CLEAN_BASELINE - CORRUPTED_BASELINE
    )

print(f"Clean Baseline is 1: {ioi_metric(clean_logits).item():.4f}")
print(f"Corrupted Baseline is 0: {ioi_metric(corrupted_logits).item():.4f}")

Clean Baseline is 1: 1.0000
Corrupted Baseline is 0: 0.0000


# 2️⃣ Attribution Patching Over Components

Intuitively, attribution patching takes a linear approximation to the model at the corrupted prompt, and calculates the effect of the local change of the patch for a single activation from corrupted to clean at that prompt.

To compute this, we take a backwards pass on the corrupted prompt with respect to the patching metric, and cache all gradients with respect to the activations. **Every single patch can be computed after caching activations from two forward passes and one backwards pass.**

In [37]:
clean_out = []
corrupted_out = []
corrupted_grads = []

with model.trace() as tracer:
    
    with tracer.invoke(clean_tokens) as invoker_clean:

        for layer in model.transformer.h:
            attn_out = layer.attn.c_proj.input[0][0]
            clean_out.append(attn_out.save())

    with tracer.invoke(corrupted_tokens) as invoker_corrupted:

        for layer in model.transformer.h:
            attn_out = layer.attn.c_proj.input[0][0]
            corrupted_out.append(attn_out.save())
            corrupted_grads.append(attn_out.grad.save())

        logits = model.lm_head.output.save()

value = ioi_metric(logits.value.cpu())
value.backward()

Then, for a given activation we can compute `(corrupted_grad_act * (clean_act - corrupted_act)).sum()`

Below, we compute differences over attention heads using `einops.reduce` to sum activations over heads.

In [29]:
patching_results = []

for corrupted_grad, corrupted, clean, layer in zip(
    corrupted_grads, corrupted_out, clean_out, range(len(clean_out))
):

    residual_attr = einops.reduce(
        corrupted_grad.value[:,-1,:] * (clean.value[:,-1,:] - corrupted.value[:,-1,:]),
        "batch (head dim) -> head",
        "sum",
        head = 12,
        dim = 64,
    )

    patching_results.append(
        residual_attr.detach().cpu().numpy()
    )

In [42]:
fig = px.imshow(
    patching_results, 
    color_continuous_scale="RdBu", 
    color_continuous_midpoint=0.0, 
    title="Patching Over Attention Heads"
)

fig.update_layout(
    xaxis_title="Head", 
    yaxis_title="Layer"
)

fig.show()

# 2️⃣ Attribution Patching Over Position

Below, we compute differences using the same outputs, instead summing over position.

In [35]:
patching_results = []

for corrupted_grad, corrupted, clean, layer in zip(
    corrupted_grads, corrupted_out, clean_out, range(len(clean_out))
):

    residual_attr = einops.reduce(
        corrupted_grad.value * (clean.value - corrupted.value),
        "batch pos dim -> pos",
        "sum",
    )

    patching_results.append(
        residual_attr.detach().cpu().numpy()
    )

In [41]:
fig = px.imshow(
    patching_results, 
    color_continuous_scale="RdBu", 
    color_continuous_midpoint=0.0, 
    title="Patching Over Position"
)

fig.update_layout(
    xaxis_title="Head", 
    yaxis_title="Layer"
)

fig.show()